<a href="https://colab.research.google.com/github/FajriAditiyaPutra/Orbit-Ventura/blob/main/Recommender%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender System
![alt text](https://miro.medium.com/max/687/1*-Jr1l2rlj9SBcCzlDHtN5g.jpeg)


In [1]:
#Import all libraries needed
import numpy as np
import pandas as pd
import pandas_profiling
from sklearn.metrics.pairwise import cosine_similarity
from urllib.request import urlopen
from zipfile import ZipFile
import os
from scipy.sparse.linalg import svds
from sklearn.feature_extraction.text import TfidfVectorizer
import time

## Content Based

In [2]:
#Download Data
!wget https://raw.githubusercontent.com/nikitaa30/Content-based-Recommender-System/master/sample-data.csv 

--2022-04-01 08:37:36--  https://raw.githubusercontent.com/nikitaa30/Content-based-Recommender-System/master/sample-data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 566516 (553K) [text/plain]
Saving to: ‘sample-data.csv’

sample-data.csv     100%[===================>] 553.24K  --.-KB/s    in 0.05s   

2022-04-01 08:37:36 (11.0 MB/s) - ‘sample-data.csv’ saved [566516/566516]



In [3]:
#data sinopsis
ds = pd.read_csv("sample-data.csv")
ds

,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."
...,...,...
495,496,Cap 2 bottoms - Cut loose from the maddening c...
496,497,Cap 2 crew - This crew takes the edge off fick...
497,498,All-time shell - No need to use that morning T...
498,499,All-wear cargo shorts - All-Wear Cargo Shorts ...


In [4]:
#Membuat Vektor representasi dari tiap item dengan TF-IDF
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')  
tfidf_matrix = tf.fit_transform(ds['description'])

#Menghitung Similarity
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in ds.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], ds['id'][i]) for i in similar_indices]

    results[row['id']] = similar_items[1:]
    
print('done!')

def item(id):
    return ds.loc[ds['id'] == id]['description'].tolist()[0].split(' - ')[0]

done!


In [5]:
# Just reads the results out of the dictionary.
def recommend(item_id_sebelumnya, num):
    print("Recommending " + str(num) + " products similar to " + item(item_id_sebelumnya) + "...")
    print("-------")
    recs = results[item_id_sebelumnya][:num]
    for rec in recs:
        print("Recommended: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

In [6]:
recommend(item_id_sebelumnya=17, num=7)

Recommending 7 products similar to Bucket hat...
-------
Recommended: Surf brim (score:0.138833717363681)
Recommended: Beach bucket (score:0.13274025336344789)
Recommended: Trim brim hat (score:0.12344537833385748)
Recommended: Baby sun bucket hat (score:0.1101353623112151)
Recommended: Baby baggies shorts (score:0.09623883085384513)
Recommended: River shorts (score:0.08177912444964454)
Recommended: Velocity visor (score:0.08136801570166713)


## Collaborative - Memory Based

In [7]:
#Download data berformat zip, lalu diunzip
!wget 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
!unzip 'ml-latest-small.zip'

--2022-04-01 08:38:22--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K   943KB/s    in 1.0s    

2022-04-01 08:38:23 (943 KB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [8]:
rating_data = pd.read_csv('ml-latest-small/ratings.csv')
movie_data = pd.read_csv('ml-latest-small/movies.csv')
rating_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
rating_data.drop('timestamp', axis = 1, inplace=True)
rating_data.head(2)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0


In [10]:
movie_data.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


Kedua file ini saat ini sudah memiliki data peringkat pengguna dan data film yang terpisah.

Namun, karena kedua file ini memiliki kesamaan movieId, keduanya dapat digabungkan berdasarkan movieId.
Mari buat pivot_table untuk data rating berdasarkan film-pengguna.

In [11]:
user_movie_rating = pd.merge(rating_data, movie_data, on = 'movieId')
user_movie_rating.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [12]:
movie_user_rating = user_movie_rating.pivot_table('rating', index = 'title', columns='userId')
user_movie_rating = user_movie_rating.pivot_table('rating', index = 'userId', columns='title')
movie_user_rating.head(2)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
user_movie_rating

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,3.5,NaN,NaN,NaN


Seperti sebelumnya, karena item-based filtering oleh karena itu kita harus mengubah satu kolom menjadi kolom Film, kolom lainnya adalah Pengguna.

Dalam keadaan ini, pertama-tama kita akan melanjutkan dengan pemfilteran kolaboratif berbasis item.

Pertama, nilai NaN diubah menjadi 0.

In [14]:
movie_user_rating.fillna(0, inplace = True)
movie_user_rating.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Saat ini, baris adalah film dan col adalah pengguna.

Dalam item-based collaborative filtering, apabila pelanggan yang membeli suatu produk juga membeli produk lainnya, misalnya: saat kita ke supermarket kita membeli pasta gigi, juga beli sabun mandi.
Dengan kata lain, pelanggan yang melihat suatu film juga melihat film lain yg serupa. basis ini didasarkan pada peringkat yang sama.
Sekarang, mari kita periksa apakah peringkatnya serupa dengan mengukur kesamaan kosinus.

In [15]:
item_based_collabor = cosine_similarity(movie_user_rating)
item_based_collabor

array([[1.        , 0.        , 0.        , ..., 0.32732684, 0.        ,
        0.        ],
       [0.        , 1.        , 0.70710678, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.70710678, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.32732684, 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [16]:
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = movie_user_rating.index, columns = movie_user_rating.index)
item_based_collabor

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.141653,0.000000,...,0.000000,0.342055,0.543305,0.707107,0.0,0.000000,0.139431,0.327327,0.000000,0.0
'Hellboy': The Seeds of Creation (2004),0.000000,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Round Midnight (1986),0.000000,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Salem's Lot (2004),0.000000,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Til There Was You (1997),0.000000,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.211467,0.216295,0.097935,0.132489,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.192259,0.000000,0.170341,0.0
xXx (2002),0.139431,0.000000,0.000000,0.000000,0.000000,0.0,0.089634,0.000000,0.276512,0.019862,...,0.069716,0.305535,0.173151,0.246482,0.0,0.192259,1.000000,0.270034,0.100396,0.0
xXx: State of the Union (2005),0.327327,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.156764,0.000000,...,0.000000,0.382543,0.177838,0.231455,0.0,0.000000,0.270034,1.000000,0.000000,0.0


Dengan cara ini, kesamaan antar item telah berhasil didapatkan.

Sekarang, kita dapat menambahkan fungsi yang merekomendasikan item yg serupa.

In [17]:
def get_item_based_collabor(title, r=6):
    print('Terima kasih sudah menonton', title, 'kami rekomendasikan anda menonton film ini:', )
    rf = item_based_collabor[title].sort_values(ascending=False)[:r+1]
    if title in rf:
      del rf[title]
    return rf

In [18]:
start = time.time()
rekomendasinya = get_item_based_collabor("'Til There Was You (1997)", 10)
end = time.time()
waktu = end - start
print(rekomendasinya)
print(waktu)

Terima kasih sudah menonton 'Til There Was You (1997) kami rekomendasikan anda menonton film ini:
title
'Salem's Lot (2004)                                  0.857493
All This, and Heaven Too (1940)                      0.857493
Absence of Malice (1981)                             0.651057
Affair to Remember, An (1957)                        0.592057
10 (1979)                                            0.552657
Best Seller (1987)                                   0.514496
Man and a Woman, A (Un homme et une femme) (1966)    0.514496
84 Charing Cross Road (1987)                         0.493049
3 Women (Three Women) (1977)                         0.474512
All That Heaven Allows (1955)                        0.450090
Name: 'Til There Was You (1997), dtype: float64
0.0067212581634521484


## Collaborative - Model Based
Disini kita akan menggunakan SVD untuk membuat rekomendasi

In [19]:
user_movie_rating.fillna(0, inplace = True)
user_movie_rating

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.5,3.5,0.0,0.0,0.0


In [20]:
#Preprocessing
R = user_movie_rating.values                  #Menyimpan angka pada Matriks
U, sigma, Vt = svds(R, k = 50)                #membuat singular value decomposition untuk menyederhanakan matriks
sigma = np.diag(sigma)                        #eigen value
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

preds_df = pd.DataFrame(all_user_predicted_ratings, columns = user_movie_rating.columns)
preds_df.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
0,-0.069287,0.018767,-0.042895,-0.038633,-0.049699,-0.007370,0.540617,-0.009627,-0.605885,0.211192,...,0.013099,-0.460020,0.019474,-0.054220,-0.021501,0.339775,-0.286041,-0.187782,1.500375,0.033463
1,-0.028678,-0.012015,-0.010785,0.000667,-0.003074,-0.007729,0.003357,0.008565,0.149107,-0.006173,...,0.004154,-0.015531,-0.027589,-0.034685,0.016193,0.059240,-0.104825,-0.010205,0.056609,-0.000368
2,0.017487,0.001239,0.004907,0.002812,0.002341,-0.001500,0.089847,-0.006716,0.027147,-0.007325,...,0.000466,0.043423,0.022705,0.022107,-0.001260,0.022077,0.001688,0.008866,0.052732,0.003761
3,-0.009402,0.007128,-0.014513,-0.004971,0.017351,-0.033052,-0.372746,0.009112,-0.227757,0.053492,...,-0.004254,-0.145385,-0.069962,-0.057734,0.001657,0.096298,-0.198194,-0.040464,0.015941,-0.005385
4,0.014022,0.004154,0.001643,-0.007565,-0.010141,-0.009996,-0.108389,0.002099,0.011011,-0.096820,...,0.004482,0.048453,0.013028,0.004971,-0.003763,-0.080241,-0.007740,0.045689,-0.091615,0.001638


In [21]:
def recommend_movies(userID, num_recommendations=5):
  user_row_number = userID - 1 
  sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False) #sorting nilai dr matriks dekomposisi
  user_data = rating_data[rating_data.userId == (userID)]
  user_full = (user_data.merge(movie_data, how = 'left', left_on = 'movieId', 
                right_on = 'movieId').sort_values(['rating'], ascending=False)) #menggabungkan dataframe rating dan list movie yang ratingnya sudah disorting
  
  recommendations = (movie_data[~movie_data['movieId'].isin(user_full['movieId'])]).merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left', left_on = 'title',
               right_on = 'title').rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :-1]
                                                                      #memfilter film yang sudah ditonton    

  return user_full, recommendations

sudah_ditonton, rekomendasi = recommend_movies(300, 10)
sudah_ditonton.head()

,userId,movieId,rating,title,genres
16,300,6016,5.0,City of God (Cidade de Deus) (2002),Action|Adventure|Crime|Drama|Thriller
11,300,2858,5.0,American Beauty (1999),Drama|Romance
29,300,112290,5.0,Boyhood (2014),Drama
27,300,109487,5.0,Interstellar (2014),Sci-Fi|IMAX
24,300,81591,5.0,Black Swan (2010),Drama|Thriller


In [22]:
rekomendasi

,movieId,title,genres
257,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3128,4226,Memento (2000),Mystery|Thriller
46,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
43,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
2359,3147,"Green Mile, The (1999)",Crime|Drama
891,1193,One Flew Over the Cuckoo's Nest (1975),Drama
97,110,Braveheart (1995),Action|Drama|War
4782,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
6987,68157,Inglourious Basterds (2009),Action|Drama|War
3625,4995,"Beautiful Mind, A (2001)",Drama|Romance


In [23]:
start = time.time()
sudah_ditonton, rekomendasi = recommend_movies(30, 10)
end = time.time()
waktu = end - start
print(rekomendasi)
print(waktu)

      movieId                             title  \
2214     2959                 Fight Club (1999)   
7671    89745              Avengers, The (2012)   
2662     3578                  Gladiator (2000)   
1492     2028        Saving Private Ryan (1998)   
506       593  Silence of the Lambs, The (1991)   
311       356               Forrest Gump (1994)   
959      1270         Back to the Future (1985)   
893      1197        Princess Bride, The (1987)   
7281    76093   How to Train Your Dragon (2010)   
255       296               Pulp Fiction (1994)   

                                         genres  
2214                Action|Crime|Drama|Thriller  
7671               Action|Adventure|Sci-Fi|IMAX  
2662                     Action|Adventure|Drama  
1492                           Action|Drama|War  
506                       Crime|Horror|Thriller  
311                    Comedy|Drama|Romance|War  
959                     Adventure|Comedy|Sci-Fi  
893     Action|Adventure|Comedy|Fantas

In [24]:
rekomendasi

,movieId,title,genres
2214,2959,Fight Club (1999),Action|Crime|Drama|Thriller
7671,89745,"Avengers, The (2012)",Action|Adventure|Sci-Fi|IMAX
2662,3578,Gladiator (2000),Action|Adventure|Drama
1492,2028,Saving Private Ryan (1998),Action|Drama|War
506,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller
311,356,Forrest Gump (1994),Comedy|Drama|Romance|War
959,1270,Back to the Future (1985),Adventure|Comedy|Sci-Fi
893,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance
7281,76093,How to Train Your Dragon (2010),Adventure|Animation|Children|Fantasy|IMAX
255,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
